In [2]:
# 安装必要库
# !pip install openai gradio

import gradio as gr
from openai import OpenAI


/usr/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

# 配置 vLLM 的 HTTP API 基础地址和 API 密钥
openai_api_base = "http://vllm-vllm.apps.cluster-q9pdq.q9pdq.sandbox1883.opentlc.com/v1"
openai_api_key = "EMPTY"  # 如果需要鉴权，替换为实际密钥

# 创建 OpenAI 客户端
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

# 定义聊天函数
def vllm_chat(user_input, history):
    """
    与 vLLM 进行对话。
    
    参数:
    - user_input: 用户输入的文本
    - history: 聊天历史记录
    
    返回:
    - 模型的回答，以及更新的聊天历史记录
    """
    # 将聊天历史转换为消息格式
    messages = [{"role": "system", "content": "You are a helpful assistant."}]
    for user_msg, bot_msg in history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": bot_msg})
    messages.append({"role": "user", "content": user_input})

    # 调用 vLLM 模型
    try:
        response = client.chat.completions.create(
            model="mistralai/Mistral-7B-Instruct-v0.2",  # 替换为实际使用的模型
            messages=messages
        )
        print(response.choices[0].message.content)
        bot_reply = response.choices[0].message.content
    except Exception as e:
        bot_reply = f"An error occurred: {str(e)}"

    # 更新聊天历史
    history.append((user_input, bot_reply))
    return bot_reply, history



In [6]:
# 构建 Gradio 界面
with gr.Blocks() as demo:
    gr.Markdown("## vLLM Chatbot with Gradio")
    
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Your Message", placeholder="Type something here...")
    clear = gr.Button("Clear Chat")
    
    # 更新对话历史
    state = gr.State([])  # 聊天历史记录

    # 定义交互
    msg.submit(vllm_chat, [msg, state], [chatbot, state])
    clear.click(lambda: ([], []), None, [chatbot, state])

# 启动 Gradio 应用
demo.launch()


/usr/local/lib/python3.12/site-packages/gradio/components/chatbot.py:225: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(
which: no node in (/bin:/home/qxu/.local/bin:/home/qxu/bin:/usr/local/bin:/usr/local/sbin:/usr/bin:/usr/sbin:/var/lib/snapd/snap/bin:/home/qxu/.local/bin:/home/qxu/bin:/usr/local/bin:/usr/local/sbin:/usr/bin:/usr/sbin:/var/lib/snapd/snap/bin)


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


ChatCompletion(id='cmpl-18bd91bf69254b548c241404468e4c59', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=' Hello! How can I help you today? If you have any questions or need assistance with something, please don\'t hesitate to ask. I\'m here to help.\n\nYou can ask me about a variety of topics, such as mathematics, science, English grammar, history, and more. I can also help you find information on the internet, set reminders, and answer general knowledge questions.\n\nIf you\'re not quite sure what you need help with, feel free to ask me anything and I\'ll do my best to provide you with accurate and helpful information.\n\nIs there a specific question or task you have in mind? Let me know and I\'ll be happy to help!\n\nHere\'s an example of a question I could help with: "What is the capital city of France?" The answer is: Paris.\n\nLet me know if you have any questions or need help with anything else!', refusal=None, role='assistan

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/gradio/blocks.py", line 2025, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/gradio/blocks.py", line 1831, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/gradio/components/chatbot.py", line 532, in postproce